In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import sys
sys.path.insert(1, 'C:/Users/pnknv/Documents/GitHub/yadisk_loader')
from main_utils import *

Необходимо создать алгоритм обновления существующих и добавления несуществующих файлов в realty </br>
В prices необходимо добавлять все пришедшие цены


# вопросы 
### 0. ЧТО НУЖНО ДОБАВИТЬ
 <b>ad_id в realty и houses в bigint</b> </br> 
 <b>добавить колонку с названиями городов из авито из ссылки</b> </br> 
 <b>добавить индекс к ad_id realty</b> </br>
 <b>добавлять темповую таблицу для обновления realty</b> </br>

### 1. Колонки 
Какие из колонок могут измениться? </br> 
Ключ: ад_айди + ссылка, все остальные на новые значения, STATUS = ? STATUS_NEW=? house_id=?
### 2. Обновление тест базы
Как часто происходи?

In [3]:
st_time = datetime.now()
print('Скрипт запущен: ', get_today_date())

Скрипт запущен:  04/04/2023, 21:08:01


In [4]:
ssh_host, ssh_port, ssh_username, ssh_password, \
database_username, database_password, database_name, \
localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config()

In [5]:
local_save_dir_avito = create_load_save_dir('avito')
local_save_dir_cian = create_load_save_dir('cian')

In [6]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost,
                                        localhost_port, database_username, database_password, database_name)

In [7]:
handled_files_avito = get_saved_files_names('avito')
handled_files_cian = get_saved_files_names('cian')

In [8]:
files_to_process_avito = ['27-03-23 (без дублей).csv']
files_to_process_cian = ['27-03-23.csv']

#### функции для новой таблицы

In [27]:
create_table_query = \
    """CREATE TABLE `temp_realty` (
      `id` int(11) NOT NULL AUTO_INCREMENT,
      `source_id` int(11) DEFAULT NULL,
      `ad_id` BIGINT DEFAULT NULL,
      `city_id` int(11) DEFAULT NULL,
      `district_id` int(11) DEFAULT NULL,
      `type_id` int(11) DEFAULT NULL,
      `addr` varchar(255) DEFAULT NULL,
      `square` float DEFAULT '0',
      `floor` int(11) DEFAULT NULL,
      `house_floors` int(11) DEFAULT NULL,
      `link` varchar(512) DEFAULT NULL,
      `date` timestamp NULL DEFAULT CURRENT_TIMESTAMP,
      `status` int(11) DEFAULT '0',
      `version` int(11) DEFAULT '0',
      `offer_from` varchar(255) DEFAULT NULL,
      `status_new` int(1) NOT NULL DEFAULT '0',
      `house_id` int(11) DEFAULT NULL,
      PRIMARY KEY (`id`)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 AVG_ROW_LENGTH=2048 ROW_FORMAT=DYNAMIC;"""

index_show_query = \
f"""SHOW indexes FROM temp_realty"""

index_create_query = \
f"""CREATE INDEX index_ad_id_temp
    ON temp_realty (ad_id);"""

clear_temp_table_query = \
    """DELETE FROM temp_realty"""

In [107]:
def get_exist_ad_id(engine, source):
    source_id = 2 if source == 'cian' else 3
    ad_id_query = f'SELECT ad_id FROM realty where source_id = {source_id}'
    try:
        con_obj = engine.connect()
        ad_id_db = pd.read_sql(text(ad_id_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        ad_id_db = None
        exc_code = exc.code
    return ad_id_db, exc_code

In [108]:
exist_ad_id, exc_code = get_exist_ad_id(sql_engine, 'avito')

In [109]:
exist_ad_id

,ad_id


In [114]:
def get_date_from_name(fname):
    try:
        return datetime.strptime(fname.replace("(без дублей)", "").replace("циан", "").replace(" ", "")[:8], "%d-%m-%y")
    except:
        return datetime.strptime(Path(fname).stem.replace("(без дублей)", "").replace("циан", "").replace(" ", "")[:8],
                                 "%d-%m-%y")

In [116]:
'27-03-23 (без дублей) (1).csv'.replace("(без дублей)", "").replace("циан", "").replace(" ", "")[:8]

'27-03-23'

In [117]:
len(["https://krasnodar.cian.ru/sale/flat/268501106/","https://krasnodar.cian.ru/sale/flat/283122800/","https://krasnodar.cian.ru/sale/flat/268652963/","https://krasnodar.cian.ru/sale/flat/269285250/","https://krasnodar.cian.ru/sale/flat/271542659/","https://krasnodar.cian.ru/sale/flat/272617108/","https://krasnodar.cian.ru/sale/flat/274448884/","https://krasnodar.cian.ru/sale/flat/274845706/","https://krasnodar.cian.ru/sale/flat/275282872/","https://krasnodar.cian.ru/sale/flat/275681690/","https://krasnodar.cian.ru/sale/flat/275683266/","https://krasnodar.cian.ru/sale/flat/275683299/","https://krasnodar.cian.ru/sale/flat/276624685/","https://krasnodar.cian.ru/sale/flat/276764488/","https://krasnodar.cian.ru/sale/flat/276987150/","https://krasnodar.cian.ru/sale/flat/277302326/","https://krasnodar.cian.ru/sale/flat/277927684/","https://krasnodar.cian.ru/sale/flat/278329463/","https://krasnodar.cian.ru/sale/flat/278603161/","https://krasnodar.cian.ru/sale/flat/279519245/","https://krasnodar.cian.ru/sale/flat/279605237/","https://krasnodar.cian.ru/sale/flat/279829358/","https://krasnodar.cian.ru/sale/flat/279989436/","https://krasnodar.cian.ru/sale/flat/280097662/","https://krasnodar.cian.ru/sale/flat/280296242/","https://krasnodar.cian.ru/sale/flat/280457302/","https://krasnodar.cian.ru/sale/flat/280460442/","https://krasnodar.cian.ru/sale/flat/280713746/","https://krasnodar.cian.ru/sale/flat/280766006/","https://krasnodar.cian.ru/sale/flat/281330536/","https://krasnodar.cian.ru/sale/flat/281445988/","https://krasnodar.cian.ru/sale/flat/282352991/","https://krasnodar.cian.ru/sale/flat/282579269/","https://krasnodar.cian.ru/sale/flat/282744947/","https://krasnodar.cian.ru/sale/flat/282760988/","https://krasnodar.cian.ru/sale/flat/282839211/","https://krasnodar.cian.ru/sale/flat/283266808/","https://krasnodar.cian.ru/sale/flat/283562495/","https://krasnodar.cian.ru/sale/flat/283567314/","https://krasnodar.cian.ru/sale/flat/283819748/","https://krasnodar.cian.ru/sale/flat/284076018/","https://krasnodar.cian.ru/sale/flat/284115904/","https://krasnodar.cian.ru/sale/flat/284128488/","https://krasnodar.cian.ru/sale/flat/284262813/","https://krasnodar.cian.ru/sale/flat/284366625/","https://krasnodar.cian.ru/sale/flat/284386276/","https://krasnodar.cian.ru/sale/flat/284496690/","https://krasnodar.cian.ru/sale/flat/284518734/","https://krasnodar.cian.ru/sale/flat/284677180/","https://krasnodar.cian.ru/sale/flat/284677421/","https://krasnodar.cian.ru/sale/flat/284704508/","https://krasnodar.cian.ru/sale/flat/275683652/","https://krasnodar.cian.ru/sale/flat/284814448/","https://krasnodar.cian.ru/sale/flat/284864849/","https://krasnodar.cian.ru/sale/flat/285115336/","https://krasnodar.cian.ru/sale/flat/285252354/"])

56

In [115]:
get_date_from_name('27-03-23 (без дублей) (1).csv')

datetime.datetime(2023, 3, 27, 0, 0)

In [30]:
def get_index_temp(engine):
    index_show_query = \
        f"""SHOW indexes FROM temp_realty"""
    try:
        con_obj = engine.connect()
        index_db = pd.read_sql(text(index_show_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        index_db = None
        exc_code = exc.code
    return index_db, exc_code

In [110]:
con_obj = sql_engine.connect()
if 'index_ad_id_temp' in get_index_temp(sql_engine)[0].Key_name.to_list():
        print('a')
# else:
#     con_obj.execute(text(index_create_query))

a


In [28]:
con_obj = sql_engine.connect()
try:
    con_obj.execute(text(create_table_query))
    if 'index_ad_id_temp' in get_index_temp(sql_engine)[0].Key_name.to_list():
        pass
    else:
        con_obj.execute(text(index_create_query))
    con_obj.commit()
    print('Временная таблица создана')
except:
    con_obj.execute(text(clear_table_query))
    con_obj.commit()
    print('Временная таблица очищена')
con_obj.close()

Временная таблица создана


In [16]:
for filename in files_to_process_cian:
    # обработка realty циан
    print('обработка файла {} для cian'.format(filename))
    df_cian_realty, file_date, error_file_processing = process_realty(local_save_dir_cian, filename,
                                                                      sql_engine, 'cian')

    # проверка состояния
    if error_file_processing:
        print('Ошибка при обработке файлов из CIAN {}. Перезапуск скрипта...'.format(filename))
        error_processing_files = True
        break
    else:
        error_processing_files = False
        print('Выгрузка в таблицу realty обработанного файла из cian:', filename)

    # загрузка realty в таблицу
    try:
#         df_cian_realty[list_realty_cols].to_sql(name='realty', con=sql_engine, if_exists='append',
#                                chunksize=7000, method='multi', index=False)
        # df_cian_realty.to_csv(f'{filename}_test_cian_realty.csv')
        error_writing_files = False
    except Exception as exc:
        print(exc)
        print('Не удалось добавить данные в таблицу {}. Перезапуск скрипта...'.format('realty'))
        error_writing_files = True
        break

обработка файла 27-03-23.csv для cian
Успешно обработан файл 27-03-23.csv для realty
Выгрузка в таблицу realty обработанного файла из cian: 27-03-23.csv


# апдейт

In [74]:
def asd():
    print('a')
    return

In [75]:
asd()

a


In [18]:
exist_ad_id = get_exist_ad_id(sql_engine, 'cian')[0].ad_id.to_list()

In [36]:
df_cian_realty_exist___ = df_cian_realty[df_cian_realty.ad_id.isin([])][list_realty_cols]

In [38]:
len(df_cian_realty_exist___)

0

In [120]:
df_cian_realty_new

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
1189,2,270328443,5.0,NaN,6,Краснодарский край; Анапа; Владимирская улица;...,95.0,20,20,https://anapa.cian.ru/sale/flat/270328443/,2023-03-27,None,274,NaN,1
3306,2,278996199,5.0,NaN,3,Краснодарский край; Анапа; Алексеевка мкр; ули...,35.0,6,7,https://anapa.cian.ru/sale/flat/278996199/,2023-03-27,None,274,Собственник,1
3433,2,279241124,5.0,NaN,2,Краснодарский край; Анапа; Таманская улица; 12...,26.0,5,6,https://anapa.cian.ru/sale/flat/279241124/,2023-03-27,None,274,Агентство недвижимости,1
3622,2,279601823,5.0,NaN,3,Краснодарский край; Анапа; улица Толстого; 26,35.5,2,7,https://anapa.cian.ru/sale/flat/279601823/,2023-03-27,None,274,Представитель застройщика,1
4159,2,280802412,5.0,NaN,3,Краснодарский край; Анапа; 18/3 ЖК,44.7,7,15,https://anapa.cian.ru/sale/flat/280802412/,2023-03-27,None,274,Застройщик,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103599,2,285370017,17.0,387.0,4,Томская область; Томск; р-н Кировский; д. Лоск...,57.1,2,4,https://tomsk.cian.ru/sale/flat/285370017/,2023-03-27,None,274,NaN,1
103600,2,285370096,17.0,405.0,4,Томская область; Томск; р-н Октябрьский; мкр. ...,53.0,8,10,https://tomsk.cian.ru/sale/flat/285370096/,2023-03-27,None,274,Агентство недвижимости,1
103601,2,285370463,17.0,417.0,4,Томская область; Томск; р-н Ленинский; мкр. Ра...,53.0,6,10,https://tomsk.cian.ru/sale/flat/285370463/,2023-03-27,None,274,Агентство недвижимости,1
103602,2,285373005,17.0,394.0,3,Томская область; Томск; р-н Ленинский; Черемош...,31.5,7,10,https://tomsk.cian.ru/sale/flat/285373005/,2023-03-27,None,274,Агентство недвижимости,1


In [19]:
df_cian_realty_exist = df_cian_realty[df_cian_realty.ad_id.isin(exist_ad_id)][list_realty_cols]
df_cian_realty_new = df_cian_realty[~df_cian_realty.ad_id.isin(exist_ad_id)][list_realty_cols]

In [24]:
df_cian_realty_exist.head(3)

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5.0,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-27,None,274,Собственник,1
1,2,213575526,5.0,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-27,None,274,Риелтор,1
2,2,232012292,5.0,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-27,None,274,Риелтор,1


In [ ]:
# обновление данных
df_cian_realty_exist.to_sql(name='temp_realty', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)

con_obj = sql_engine.connect()
common_ids = tuple(df_cian_realty_exist.ad_id)
update_table_query = f"""update realty join temp_realty on realty.ad_id=temp_realty.ad_id
                        set realty.source_id = temp_realty.source_id,
                            realty.city_id = temp_realty.city_id,
                            realty.district_id = temp_realty.district_id,
                            realty.type_id = temp_realty.type_id,
                            realty.addr = temp_realty.addr,
                            realty.square = temp_realty.square,
                            realty.floor = temp_realty.floor,
                            realty.house_floors = temp_realty.house_floors,
                            realty.link = temp_realty.link,
                            realty.date = temp_realty.date,
                            realty.status = temp_realty.status,
                            realty.version = temp_realty.version,
                            realty.offer_from = temp_realty.offer_from,
                            realty.status_new = temp_realty.status_new,
                            realty.house_id = temp_realty.house_id
                         WHERE realty.ad_id in {common_ids}"""
con_obj.execute(text(update_table_query))
con_obj.commit()
con_obj.close()

In [ ]:
# очистка данных из temp_realty
con_obj = sql_engine.connect()
con_obj.execute(text(clear_temp_table_query))
con_obj.commit()
con_obj.close()

In [ ]:
# добавление данных 
df_cian_realty_new.to_sql(name='realty', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)